In [1]:
#import os
import tensorflow as tf
import pandas as pd
import numpy as np
#from sklearn import svm
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler
from sklearn.pipeline import Pipeline
from categorical_encoder import CategoricalEncoder,DataFrameSelector
from sklearn.pipeline import FeatureUnion

In [2]:
from sklearn.model_selection import train_test_split

In [29]:
from sklearn.metrics import classification_report,accuracy_score

In [3]:
df = pd.read_csv('loan_problem_3.csv')
df = df.dropna()
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
5,LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y


In [4]:
df.columns

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

In [5]:
df['Loan_Status'] = df['Loan_Status'].apply(lambda x: 1 if x == 'Y' else 0 )
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,0
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,1
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,1
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,1
5,LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,1


In [6]:
numeric_columns =['ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term','Credit_History']
feature_columns = 

In [7]:
df[numeric_columns] = df[numeric_columns].apply(lambda x: (x - x.min())/(x.max() - x.min()))

In [9]:
X = df.drop(['Loan_ID','Loan_Status'],axis=1)
y = df['Loan_Status']

In [11]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 480 entries, 1 to 613
Data columns (total 11 columns):
Gender               480 non-null object
Married              480 non-null object
Dependents           480 non-null object
Education            480 non-null object
Self_Employed        480 non-null object
ApplicantIncome      480 non-null float64
CoapplicantIncome    480 non-null float64
LoanAmount           480 non-null float64
Loan_Amount_Term     480 non-null float64
Credit_History       480 non-null float64
Property_Area        480 non-null object
dtypes: float64(5), object(6)
memory usage: 45.0+ KB


In [12]:
gender_feature = tf.feature_column.categorical_column_with_hash_bucket('Gender',hash_bucket_size=3)
married_feature = tf.feature_column.categorical_column_with_hash_bucket('Married',hash_bucket_size=10)
dependents_feature = tf.feature_column.categorical_column_with_hash_bucket('Dependents',hash_bucket_size=10)
education_feature = tf.feature_column.categorical_column_with_hash_bucket('Education',hash_bucket_size=10)
self_employed_feature = tf.feature_column.categorical_column_with_hash_bucket('Self_Employed',hash_bucket_size=10)
applicant_income_feature = tf.feature_column.numeric_column('ApplicantIncome')
coapplicant_income_feature = tf.feature_column.numeric_column('CoapplicantIncome')
loan_amount_feature = tf.feature_column.numeric_column('LoanAmount')
loan_amount_term_feature = tf.feature_column.numeric_column('Loan_Amount_Term')
credit_history_feature = tf.feature_column.numeric_column('Credit_History')
property_area_feature = tf.feature_column.categorical_column_with_hash_bucket('Property_Area',hash_bucket_size=10)

In [13]:
tf_feature_list =[
gender_feature,
married_feature,
dependents_feature,
education_feature,
self_employed_feature,
applicant_income_feature,
coapplicant_income_feature,
loan_amount_feature,
loan_amount_term_feature,
credit_history_feature,
property_area_feature    
]

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=101)

In [15]:
train_input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=10,num_epochs=1000,shuffle=True)

In [16]:
lc_model = tf.estimator.LinearClassifier(feature_columns=tf_feature_list,n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_model_dir': '/var/folders/mk/6ts23z1s0c53hhcxrsbkwjhm0000gn/T/tmpg8kkza6f', '_save_summary_steps': 100, '_log_step_count_steps': 100, '_tf_random_seed': 1, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000}


In [70]:
lc_model.train(input_fn=train_input_func,steps=10000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /var/folders/mk/6ts23z1s0c53hhcxrsbkwjhm0000gn/T/tmpg8kkza6f/model.ckpt-21100
INFO:tensorflow:Saving checkpoints for 21101 into /var/folders/mk/6ts23z1s0c53hhcxrsbkwjhm0000gn/T/tmpg8kkza6f/model.ckpt.
INFO:tensorflow:step = 21101, loss = 4.27742
INFO:tensorflow:global_step/sec: 124.689
INFO:tensorflow:step = 21201, loss = 5.14137 (0.803 sec)
INFO:tensorflow:global_step/sec: 130.879
INFO:tensorflow:step = 21301, loss = 5.09145 (0.764 sec)
INFO:tensorflow:global_step/sec: 132.277
INFO:tensorflow:step = 21401, loss = 7.51538 (0.756 sec)
INFO:tensorflow:global_step/sec: 138.292
INFO:tensorflow:step = 21501, loss = 2.60291 (0.723 sec)
INFO:tensorflow:global_step/sec: 137.861
INFO:tensorflow:step = 21601, loss = 2.50459 (0.726 sec)
INFO:tensorflow:global_step/sec: 131.958
INFO:tensorflow:step = 21701, loss = 5.11517 (0.757 sec)
INFO:tensorflow:global_step/sec: 139.172
INFO:tensorflow:step = 21801, loss = 5.

INFO:tensorflow:step = 29201, loss = 5.18261 (0.738 sec)
INFO:tensorflow:global_step/sec: 135.329
INFO:tensorflow:step = 29301, loss = 4.66549 (0.739 sec)
INFO:tensorflow:global_step/sec: 138.344
INFO:tensorflow:step = 29401, loss = 3.02095 (0.723 sec)
INFO:tensorflow:global_step/sec: 144.13
INFO:tensorflow:step = 29501, loss = 3.7211 (0.694 sec)
INFO:tensorflow:global_step/sec: 134.254
INFO:tensorflow:step = 29601, loss = 2.93614 (0.745 sec)
INFO:tensorflow:global_step/sec: 138.105
INFO:tensorflow:step = 29701, loss = 2.2464 (0.724 sec)
INFO:tensorflow:global_step/sec: 132.26
INFO:tensorflow:step = 29801, loss = 4.31845 (0.757 sec)
INFO:tensorflow:global_step/sec: 138.602
INFO:tensorflow:step = 29901, loss = 4.50672 (0.721 sec)
INFO:tensorflow:global_step/sec: 135.783
INFO:tensorflow:step = 30001, loss = 4.5378 (0.736 sec)
INFO:tensorflow:global_step/sec: 135.338
INFO:tensorflow:step = 30101, loss = 2.85724 (0.739 sec)
INFO:tensorflow:global_step/sec: 132.201
INFO:tensorflow:step = 30

In [71]:
test_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=10,num_epochs=1,shuffle=False)

In [72]:
preds = list(lc_model.predict(input_fn=test_input_func))

INFO:tensorflow:Restoring parameters from /var/folders/mk/6ts23z1s0c53hhcxrsbkwjhm0000gn/T/tmpg8kkza6f/model.ckpt-31100


In [73]:
y_preds = [item['class_ids'][0] for item in preds]

In [74]:
print(classification_report(y_test,y_preds))

             precision    recall  f1-score   support

          0       0.83      0.34      0.48        44
          1       0.77      0.97      0.86       100

avg / total       0.79      0.78      0.74       144



In [75]:
print(accuracy_score(y_test,y_preds))

0.777777777778


5000 Steps:
  Accuracy: 0.78, Precision: 0.79, Recall: 0.78, F1: 0.74

1000 steps:
  Accuracy: 0.78, Precision: 0.81, Recall: 0.78, F1: 0.75